In [17]:
import os
from typing import Optional, TypedDict
from langchain_core.documents.base import Document
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain_community.document_loaders import PyPDFLoader, UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.vectorstores.base import VectorStoreRetriever
from langchain_community.vectorstores import FAISS
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
import dotenv

In [18]:
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

In [19]:
# .env파일 읽기
dotenv.load_dotenv()

# os라이브러리 이용한 읽기
# os.environ['OPENAI_API_KEY'] = "API-Key"

True

In [23]:
class State(TypedDict):
    query : str
    response : str
    document : Document   

In [21]:
llm = ChatOpenAI(model="gpt-4o-mini",
                 temperature=0.0,)

In [ ]:
def chatbot(llm, query: State["query"], state: State) -> State["message"]:

    return {"message": llm.invoke(state["query"]).content}

In [6]:
prompt = ChatPromptTemplate.from_messages([
    ("system", 
     """
    context : {context}

    당신은 언제나 고객에게 최선을 다해 답변을 하며 말투는 굉장히 친근합니다. 직업은 전문 상담원입니다. 답변 시, 아래의 규칙을 지켜야만 합니다.
    규칙 1. 주어진 context만을 이용하여 답변해야합니다. 
    규칙 2. 주어진 context에서 답변을 할 수 없다면 "해당 문의는 010-2255-3366으로 연락주시면 도와드리겠습니다. 영업 시간은 오전 10시-오후 6시입니다." 라고 대답하세요.
    규칙 3. 문자열에 A1, A2, A11, A22 등 필요 없는 문자는 제거한 뒤 출력합니다.
    규칙 4. 항상 친절한 말투로 응대합니다.
    규칙 5. 웹사이트 링크를 그대로 출력합니다. 대소문자를 명확하게 구분하세요.
    """),
    ("human", "{query}")
])

In [22]:
def embedding_file(file: str) -> VectorStoreRetriever:
    splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=300,
        chunk_overlap=100,
        separators=["\n\n"],
    )

    loader = PyPDFLoader(f"./files/{file}")
    docs = loader.load_and_split(text_splitter=splitter)

    embeddings = OpenAIEmbeddings()
    vector_store = FAISS.from_documents(docs, embeddings)
    retriever = vector_store.as_retriever()    

    return retriever

In [7]:
def format_docs(docs: list[Document]) -> str:
    return "\n\n".join(doc.page_content for doc in docs)